# Init

In [1]:
!pip install -qq pysastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 7.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

import nltk
from nltk.tokenize import word_tokenize 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.parse.transitionparser import remove
from collections import defaultdict

def lower_text(texts):
    return [s.lower() for s in texts]

def remove_punc_text(texts):
    return [s.translate(str.maketrans("","",string.punctuation + "1234567890")) for s in texts]

def strip_text(texts):
    return [s.strip() for s in texts]

def remove_stopword_text(texts):
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()
    stopwords.remove('mata')
    stopwords.remove('ingat')
    stopwords.remove('orang')
    ext_stopwords = ['dok', 'doc', 'dokter', 'terima', 'kasih', 'terimakasih']
    all_stopwords = stopwords + ext_stopwords
    dictionary = ArrayDictionary(all_stopwords)
    stopword = StopWordRemover(dictionary)
    return [stopword.remove(s) for s in texts]

def stemming_text(texts):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return [stemmer.stem(s) for s in texts]

def tokenize_text(texts):
    return [word_tokenize(s) for s in texts]

def cleaning_text(texts):
    texts = lower_text(texts)
    texts = remove_punc_text(texts)
    texts = strip_text(texts)
    texts = remove_stopword_text(texts)
    texts = stemming_text(texts)
    return texts

def cleaning_text_tokenize(texts):
    texts = lower_text(texts)
    texts = remove_punc_text(texts)
    texts = strip_text(texts)
    texts = remove_stopword_text(texts)
    texts = stemming_text(texts)
    texts = tokenize_text(texts)
    return texts

def cleaning_text_stemming(texts):
    texts = lower_text(texts)
    texts = remove_punc_text(texts)
    texts = strip_text(texts)
    texts = stemming_text(texts)
    return texts

def cleaning_text_raw(texts):
    texts = lower_text(texts)
    texts = remove_punc_text(texts)
    texts = strip_text(texts)
    return texts

In [4]:
categories = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi', 'Umum']

categories_new = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi']





In [5]:
df = pd.read_csv('Dataset/Human_Annotated.csv', index_col='ID')
df.fillna("", inplace = True)

df_umumnt = df.drop(columns=["Umum"])
df_umumnt["COUNT CLASS"] = df_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df = df_umumnt[df_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])

In [6]:
df['ALL'] = df['JUDUL'] + ' ' + df['ISI']
# label = df[categories]
label = df[categories_new]
df = df[["ALL"]]

In [7]:
df.head()

,ALL
ID,
DS-1,mengapa keringat badan sangat berlebihan ? sel...
DS-23,suka menghayal dan berhalusinasi selamat siang...
DS-87,feses warna kuning di sertai darah apakah ada ...
DS-176,apakah setiap nyeri perut kanan bawah itu adal...
DS-277,10 kebiasaan buruk yang dapat merusak otak say...
...,...
KD-6692,luka di sekitar kelamin gimana cara menghilang...
DH-2238,benjol-benjol kecil di buah zakar dok saya lak...
KD-37628,benjolan kandung kemih salam dok . ayah saya d...


In [8]:
k_label = label.sum()

In [9]:
corpus = cleaning_text(df['ALL'].values)

# Class untuk menghitung skor

In [33]:
class WordCounter:
  def __init__(self, category):
    self.unigram = {}
    self.total_uni = np.array([0] * len(category))
    self.bigram = {}
    self.total_bi = np.array([0] * len(category))
    self.trigram = {}
    self.total_tri = np.array([0] * len(category))
    self.token_count = {}
    self.category = category

  def add(self, token, labels):
    vocab = self.get_ngram(token)
    if token not in vocab:
      vocab[token] = np.array([0] * len(self.category))
      self.token_count[token] = 0
    vocab[token] = np.sum([vocab[token], labels], axis=0)
    self.token_count[token] += 1
    self.set_total(token, np.sum([self.get_total(token), labels], axis=0))

  def get_ngram(self, token):
    n = len(token.split())
    if n == 1:
      return self.unigram
    elif n == 2:
      return self.bigram
    elif n == 3:
      return self.trigram
    else:
      return None

  def get_total(self, token):
    n = len(token.split())
    if n == 1:
      return self.total_uni
    elif n == 2:
      return self.total_bi
    elif n == 3:
      return self.total_tri
    else:
      return None

  def set_total(self, token, new_total):
    n = len(token.split())
    if n == 1:
      self.total_uni = new_total 
    elif n == 2:
      self.total_bi = new_total
    elif n == 3:
      self.total_tri = new_total
    else:
      return None

  def get_word_count(self, token):
    n = len(token.split())
    if n == 1:
      return self.unigram[token]
    elif n == 2:
      return self.bigram[token]
    elif n == 3:
      return self.trigram[token]
    else:
      return None

  def get_token(self, token, normalize = ""):
    try:
      word_count = self.get_word_count(token)
    except KeyError:
      print(f"{token} not found")
      return None
    if normalize == "class":
      norm = np.divide(word_count, self.get_total(token))
    elif normalize == "token":
      norm = np.divide(word_count, self.token_count[token])
    elif normalize == "class_token":
      norm = np.divide(word_count, self.get_total(token))
      norm = np.divide(norm, self.token_count[token])
    else:
      return word_count, tuple(zip(self.category, word_count))
    return norm, tuple(zip(self.category, norm))

  def get_all_token_from_label(self, label, normalize = "", threshold = [0, 0, 0]):
    idx = self.category.index(label)
    unigram = self.unigram.keys()
    unires = {}
    bigram = self.bigram.keys()
    bires = {}
    trigram = self.trigram.keys()
    trires = {}
    for i in unigram:
      word_count = self.get_word_count(i)
      if (word_count[idx] >= threshold[0]):
        val, _ = self.get_token(i, normalize)
        unires[i] = val
    for i in bigram:
      word_count = self.get_word_count(i)
      if (word_count[idx] >= threshold[1]):
        val, _ = self.get_token(i, normalize)
        bires[i] = val
    for i in trigram:
      word_count = self.get_word_count(i)
      if (word_count[idx] >= threshold[2]):
        val, _ = self.get_token(i, normalize)
        trires[i] = val
    return unires, bires, trires
      
  def get_vocabulary(self):
    return self.vocab.keys()

  def to_dict(self):
    memo = {}
    memo["unigram"] = self.unigram
    memo["bigram"] = self.bigram
    memo["trigram"] = self.trigram
    memo["total_token"] = self.total_token
    memo["token_count"] = self.token_count
    return memo

  def load(self, memo):
    self.unigram = memo["unigram"]
    self.bigram = memo["bigram"]
    self.trigram = memo["trigram"]
    self.total_token = memo["total_token"]
    self.token_count = memo["token_count"]

In [11]:
X = cleaning_text_tokenize(corpus)
y = label.values

In [12]:
def init():
  word_counter = WordCounter(categories_new)
  for unigram, l in zip(X, y):
    bigram = list(nltk.bigrams(unigram))
    trigram = list(nltk.trigrams(unigram))
    unigram = list(unigram)

    for i in unigram:
      word_counter.add(i, l)

    for i in bigram:
      word_counter.add(' '.join(i), l)

    for i in trigram:
      word_counter.add(' '.join(i), l)
  return word_counter

In [34]:
word_count = init()

In [ ]:
memo = word_count.to_dict()
word_count = WordCounter(categories)
word_count.load(memo)

**Cara menghitung skor keyword**

(Banyak keyword i pada label C) 


---


(Banyak keyword pada label C) * (Banyak data yang memiliki keyword i)

**Cara mengecek distribusi yang dominan**
1. scale dengan log_10
2. mencari nilai maks dari skor
3. Jika ada 1-3 label yang memiliki nilai yang sama dengan maks maka menjadi keyword
3. Jika lebih dari 3 label, maka tidak menjadi keyword karena terlalu umum

In [ ]:
(word_count.get_token('rumah', normalize = "class_token")[1])

(('Anak', 7.16563362115295e-06),
 ('Bedah', 3.7540355882573768e-06),
 ('Gigi', 2.0012007204322592e-05),
 ('Gizi', 3.7864445285876562e-06),
 ('Jantung', 1.8264284192262032e-05),
 ('Jiwa', 1.6950589032968897e-05),
 ('Kandungan', 7.498177005715486e-06),
 ('Kulit dan Kelamin', 2.8142219520569148e-06),
 ('Mata', 0.0),
 ('Paru', 1.3125082031762699e-05),
 ('Penyakit Dalam', 5.685674587522076e-06),
 ('Saraf', 8.051335313961821e-06),
 ('THT', 9.203336209375898e-06),
 ('Tulang', 2.6907760198041117e-06),
 ('Urologi', 0.0))

In [ ]:
lst = (word_count.get_token('sakit', normalize = "class_token")[0])
print(np.log10(lst).round())
word_count.get_token('sakit', normalize = "class_token")[1]

[-6. -5. -5. -6. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.]


(('Anak', 1.5529385800002704e-06),
 ('Bedah', 1.0164912788494008e-05),
 ('Gigi', 6.377954523653538e-06),
 ('Gizi', 2.7031514884710587e-06),
 ('Jantung', 1.1020987374915674e-05),
 ('Jiwa', 7.076961906712892e-06),
 ('Kandungan', 4.41619684354338e-06),
 ('Kulit dan Kelamin', 4.649584094702729e-06),
 ('Mata', 4.708496953602471e-06),
 ('Paru', 4.685013178272783e-06),
 ('Penyakit Dalam', 1.1769334951576117e-05),
 ('Saraf', 1.613508341099112e-05),
 ('THT', 8.858149350744583e-06),
 ('Tulang', 7.615199444131604e-06),
 ('Urologi', 7.881127043767946e-06))

In [15]:
uni_t = np.array(label.astype(bool).sum() * 0.1)
bi_t = np.array(label.astype(bool).sum() * 0.05)
bi_t[bi_t < 3] = 3
tri_t = np.array(label.astype(bool).sum() * 0.025)
tri_t[tri_t < 3] = 3

def condition(lst, idx, thres, strict):
  lst = np.log10(lst).round()
  if lst[idx] == lst.max() and np.count_nonzero(lst == lst.max()) <= thres and (not strict or np.count_nonzero(lst == 0) != 15):
    return True
  return False

def filter_distribution(scores, idx, thres, strict):
  filtered_scores = {}
  for i, k in scores.items():
    if condition(k, idx, thres, strict):
      filtered_scores[i] = k
  return filtered_scores

def filter_ngram(unigram, bigram, trigram, idx, strict):
  return filter_distribution(unigram, idx, 3, strict), \
  filter_distribution(bigram, idx, 2, strict), \
  filter_distribution(trigram, idx, 1, strict)

def get_keyword(label, strict):
  idx = categories.index(label)
  uni, bi, tri = word_count.get_all_token_from_label(label, 'class_token', [uni_t[idx], bi_t[idx], tri_t[idx]])
  uni, bi, tri = filter_ngram(uni, bi, tri, idx, strict)
  keyword = []
  for i in [uni, bi, tri]:
    for j in i:
      keyword.append(j)
  return keyword

In [38]:
#@title Keyword Tanpa Umum
for i in categories_new:
  print(i)
  print(get_keyword(i, True))
  print()


Anak


<ipython-input-15-e65fb7777d04>:8: RuntimeWarning: divide by zero encountered in log10
  lst = np.log10(lst).round()


['umur', 'bicara', 'bab', 'anak', 'usia', 'alergi', 'minum', 'demam', 'mimisan', 'coba', 'bayi', 'asi', 'susu', 'formula', 'ganti', 'lahir', 'sapi', 'kedelai', 'soya', 'imunisasi', 'umur tahun', 'anak usia', 'usia tahun', 'anak anak', 'anak umur', 'bayi minum', 'minum asi', 'minum susu', 'susu formula', 'formula anak', 'ganti susu', 'bayi usia', 'alergi susu', 'anak alergi', 'susu sapi', 'asi susu', 'tahun anak', 'anak makan', 'susu anak', 'susu kedelai', 'sapi anak', 'imunisasi anak', 'anak susu', 'anak minum', 'formula asi', 'badan anak', 'susu soya', 'minum asi minum', 'asi minum susu', 'minum susu formula', 'susu formula anak', 'susu formula bayi', 'pilih susu formula', 'susu formula susu', 'susu formula coba', 'bayi alergi susu', 'anak alergi susu', 'alergi susu sapi', 'asi susu formula', 'umur tahun anak', 'anak susu formula', 'bayi minum susu', 'anak minum susu', 'susu formula asi', 'berat badan anak', 'konsumsi susu formula', 'anak konsumsi susu', 'minum susu soya']

Bedah
['pe

# Keyword yang sudah dicek kembali secara manual

In [ ]:
Anak
['anak', 'alergi', 'minum', 'demam', 'mimisan', 'coba', 'bayi', 'asi', 'susu', 'formula', 'ganti', 'lahir', 'sapi', 'kedelai', 'soya', 'imunisasi', 'umur tahun', 'anak usia', 'usia tahun', 'anak anak', 'anak umur', 'bayi minum', 'minum asi', 'minum susu', 'susu formula', 'formula anak', 'ganti susu', 'bayi usia', 'alergi susu', 'anak alergi', 'susu sapi', 'asi susu', 'tahun anak', 'anak makan', 'susu anak', 'susu kedelai', 'sapi anak', 'imunisasi anak', 'anak susu', 'anak minum', 'formula asi', 'badan anak', 'susu soya', 'minum asi minum', 'asi minum susu', 'minum susu formula', 'susu formula anak', 'susu formula bayi', 'pilih susu formula', 'susu formula susu', 'susu formula coba', 'bayi alergi susu', 'anak alergi susu', 'alergi susu sapi', 'asi susu formula', 'umur tahun anak', 'anak susu formula', 'bayi minum susu', 'anak minum susu', 'susu formula asi', 'berat badan anak', 'konsumsi susu formula', 'anak konsumsi susu', 'minum susu soya']

Bedah
['perut', 'usus', 'buntu', 'kembung', 'benjol', 'anus', 'wasir', 'bab', 'usg', 'bekas', 'operasi', 'jahit', 'selamat sore', 'nyeri perut', 'perut kanan', 'usus buntu', 'perut kembung', 'sakit usus', 'gejala usus', 'operasi usus', 'obat wasir', 'buntu operasi', 'bab darah', 'luka operasi', 'bekas operasi', 'nyeri perut kanan', 'usus buntu siang', 'perut belah kanan', 'operasi usus buntu', 'usus buntu operasi', 'wasir obat wasir', 'pasca operasi usus']

Gigi
['siang', 'warna', 'hilang', 'alami', 'sehat', 'tumbuh', 'kanker', 'periksa', 'minggu', 'bersih', 'ubah', 'demam', 'sariawan', 'bau', 'lidah', 'gigi', 'lubang', 'mutih', 'geraham', 'gusi', 'tambal', 'ngilu', 'karang', 'cabut', 'pasta', 'sikat', 'selamat siang', 'gigi gigi', 'gigi lubang', 'gigi geraham', 'ngilu ganggu', 'gigi tambal', 'saraf gigi', 'alami sariawan', 'tambal gigi', 'gigi bersih', 'karang gigi', 'gigi ngilu', 'alami ngilu', 'gigi darah', 'gusi bengkak', 'obat gigi', 'pasta gigi', 'cabut gigi', 'sikat gigi', 'derita sariawan', 'sariawan minggu', 'sariawan sembuh', 'gigi sakit', 'gigi kuning', 'mutih gigi', 'gigi warna', 'gigi putih', 'putih gigi', 'kuning gigi', 'gusi darah', 'darah sariawan', 'gigi patah', 'warna gigi', 'bau mulut', 'gigi warna kuning', 'gusi darah sariawan', 'warna kuning gigi']

Gizi
['badan', 'milik', 'sore', 'berat', 'saran', 'sehat', 'kurus', 'hamil', 'makan', 'asam', 'lambung', 'isi', 'konsumsi', 'cepat', 'gemuk', 'susu', 'program', 'tubuh', 'tinggi', 'kg', 'kedelai', 'ideal', 'perempuan', 'berat badan', 'makan derita', 'derita asam', 'makan makan', 'minum susu', 'program hamil', 'tubuh kurus', 'badan kg', 'susu kedelai', 'tahun tinggi', 'badan tinggi', 'porsi makan', 'perempuan tahun', 'badan makan', 'tinggi badan', 'badan berat', 'badan kurus', 'makan berat', 'milik berat', 'badan ideal', 'kurus berat badan', 'pantang makan derita', 'makan derita asam', 'derita asam lambung', 'berat badan kg', 'air kelapa muda', 'tinggi berat badan', 'berat badan berat', 'makan berat badan', 'milik berat badan', 'susah berat badan', 'cepat berat badan', 'berat badan ideal', 'keluh berat badan', 'berat badan milik', 'berat badan selamat', 'badan tinggi badan', 'gimana berat badan', 'derita batu ginjal', 'berat badan capai', 'badan capai kg']

Jantung
['keringat', 'milik', 'orang', 'sakit', 'sehat', 'pikir', 'waktu', 'tengah', 'riwayat', 'dada', 'asam', 'lambung', 'cepat', 'panik', 'normal', 'cemas', 'jantung', 'maag', 'tinggi', 'pusar', 'lemas', 'kaku', 'rs', 'ekg', 'dosis', 'bisoprolol', 'panic', 'hipertiroid', 'mghari', 'rekam', 'mmhg', 'aritmia', 'umur tahun', 'nyeri dada', 'dada tengah', 'serang jantung', 'jantung normal', 'sakit dada', 'dada belah', 'jantung jantung', 'tekan darah', 'debar jantung', 'akibat jantung', 'dosis mghari', 'obatobat an', 'tinggi keluh', 'rekam jantung', 'nyaman dada', 'jantung akibat', 'jantung cepat', 'obat turun', 'turun tekan', 'daerah pusar', 'dada kiri nyeri', 'dada belah kiri', 'tinggi keluh sakit', 'keluh sakit dada', 'obat turun tekan', 'turun tekan darah', 'detak jantung cepat']

Jiwa
['milik', 'suka', 'bicara', 'orang', 'temu', 'pikir', 'bingung', 'dada', 'tidur', 'konsultasi', 'sulit', 'cepat', 'panik', 'depresi', 'sedih', 'bunuh', 'aktivitas', 'cemas', 'jantung', 'ganggu', 'takut', 'kakak', 'anxiety', 'disorder', 'tinggal', 'keluarga', 'mudah', 'hidup', 'cerita', 'jiwa', 'nama', 'psikiater', 'konsentrasi', 'benci', 'kantor', 'panic', 'alami depresi', 'sulit tidur', 'depresi berat', 'obat depresi', 'ganggu jiwa', 'social anxiety', 'jam pagi', 'efek samping pusing', 'suka banting barang', 'orang mudah kaget', 'disorder fobia sosial']

Kandungan
['usia', 'minggu', 'hamil', 'tanggal', 'haid', 'meni', 'flek', 'suami', 'kandung', 'usg', 'kista', 'rahim', 'indung', 'telur', 'coklat', 'hasil', 'operasi', 'caesar', 'istri', 'normal', 'atur', 'menstruasi', 'program', 'mens', 'hubung', 'lahir', 'kb', 'siklus', 'janin', 'perempuan', 'tanda hamil', 'indung telur', 'operasi caesar', 'haid atur', 'usia kandung', 'program hamil', 'meni tahun', 'hamil minggu', 'siklus haid', 'usia hamil', 'perempuan tahun', 'flek hamil', 'badan janin', 'berat janin', 'pasca operasi caesar', 'makan mie instan', 'usia hamil minggu', 'berat badan janin']

Kulit dan Kelamin
['keringat', 'warna', 'merah', 'hilang', 'timbul', 'benjol', 'jerawat', 'muka', 'tumbuh', 'jari', 'bahaya', 'bersih', 'tahi', 'lalat', 'gatal', 'bintikbintik', 'bekas', 'tangan', 'kulit', 'muncul', 'coba', 'kelupas', 'pakai', 'wajah', 'bintik', 'telapak', 'kering', 'kelamin', 'garuk', 'produk', 'kutil', 'tahi lalat', 'tangan kaki', 'kulit kelupas', 'wajah jerawat', 'telapak tangan', 'hilang tahi', 'muncul bintikbintik', 'pakai produk', 'kutil kelamin', 'bekas jerawat', 'telapak tangan kaki', 'hilang tahi lalat', 'tahi lalat hilang']

Mata
['warna', 'merah', 'periksa', 'samping', 'ubah', 'air', 'kirakira', 'efek', 'mata', 'ganggu', 'minus', 'pakai', 'silinder', 'kacamata', 'hijau', 'belek', 'softlens', 'sakit mata', 'mata air', 'mata minus', 'mata belah', 'kiri minus', 'mata kiri', 'mata anak', 'minus silinder', 'silinder minus', 'pakai kacamata', 'mata mata', 'minus mata', 'air belek', 'mata normal', 'tetes mata', 'merah air', 'obat mata', 'mata silinder', 'kanan silinder', 'sembuh mata', 'mata merah', 'silinder sembuh', 'mata kanan', 'silinder mata', 'mata ubah', 'ganggu mata', 'merah belek', 'buta warna', 'air mata', 'mata belek', 'belek selamat', 'belek mata', 'minus silindris', 'silinder pakai', 'tangan mata', 'periksa mata', 'pakai softlens', 'softlens minus', 'mata minus silinder', 'silinder minus silinder', 'minus silinder minus', 'obat tetes mata', 'obat mata silinder', 'mata silinder adik', 'mata belah kanan', 'mata silinder sembuh', 'minus silinder mata', 'silinder mata silinder', 'mata merah belek', 'mata ubah warna', 'mata anak belek', 'mata minus silindris', 'minus silinder pakai', 'bangun tidur mata', 'merah bangun tidur', 'pakai softlens minus', 'mata merah air', 'merah air belek', 'mata silinder mata', 'belek bangun tidur']

Paru
['bantu', 'sore', 'alami', 'rokok', 'tanggal', 'keluh', 'samping', 'kemarin', 'obat', 'tenggorok', 'tidur', 'hasil', 'konsumsi', 'efek', 'tolong', 'asap', 'batuk', 'atur', 'henti', 'hidung', 'dahak', 'bronkitis', 'kental', 'flu', 'ingus', 'asma', 'lemas', 'seringkali', 'debu', 'sumbat', 'paru', 'puskesmas', 'tbc', 'fdc', 'tb', 'fibrosis', 'flu batuk', 'batuk flu', 'asma kambuh', 'samping obat', 'hidung sumbat', 'batuk darah', 'makan obat', 'obat tbc', 'luka operasi', 'efek samping obat']

Penyakit Dalam
['sakit', 'perut', 'kembung', 'kadang', 'dada', 'makan', 'asam', 'lambung', 'mual', 'ulu', 'hati', 'jantung', 'leher', 'maag', 'lemas', 'perut kanan', 'perut kembung', 'nyeri dada', 'sakit ulu', 'ulu hati', 'hati sakit', 'nyeri ulu', 'badan lemas', 'ulu hati sakit', 'nyeri ulu hati']

Saraf
['sakit', 'otak', 'keluh', 'kepala', 'tangan', 'mual', 'kirakira', 'normal', 'hidung', 'leher', 'motor', 'sepeda', 'ekor', 'stroke', 'sakit kepala', 'sakit tulang', 'kepala belah', 'kepala bentur', 'sakit leher', 'bentur kepala', 'cedera kepala', 'akibat bentur', 'sakit tulang punggung', 'sakit kepala pusing', 'tulang ekor duduk', 'kolesterol tinggi stroke']

THT
['bersih', 'obat', 'tenggorok', 'tidur', 'kepala', 'air', 'mual', 'demam', 'mimisan', 'hidung', 'flu', 'ingus', 'telinga', 'dengung', 'kotor', 'dengar', 'tht', 'gendang', 'sakit kepala', 'telinga belah', 'dengung telinga', 'hidung belah', 'telinga kiri', 'dengar telinga', 'telinga kanan', 'mimisan mimisan', 'darah hidung', 'gendang telinga', 'telinga air', 'kotor telinga', 'telinga dengung', 'telinga sakit', 'tetes telinga', 'sakit telinga', 'telinga belah kanan', 'dengung telinga belah', 'hidung belah kanan', 'gendang telinga pecah', 'telinga belah kiri', 'obat tetes telinga', 'belah kiri dengung', 'telinga dengung telinga', 'dengung telinga dengung']

Tulang
['bantu', 'tangan', 'gerak', 'tulang', 'lutut', 'patah', 'motor', 'celaka', 'tinggi', 'urut', 'bawa', 'jatuh', 'main', 'telapak', 'kering', 'retak', 'sepeda', 'cedera', 'sendi', 'tekuk', 'ligamen', 'selamat malam', 'selamat siang', 'sakit tulang', 'patah tulang', 'tulang kering', 'proses sembuh', 'tulang patah', 'akibat celaka', 'alami patah', 'jalan normal', 'tahun tinggi', 'lutut kanan', 'celaka motor', 'tinggi badan', 'gelang kaki', 'kering kanan', 'bentur separator', 'separator busway', 'motor jatuh', 'kanan bentur', 'nyeri tulang', 'tulang dada', 'tulang tulang', 'cedera lutut', 'lutut kiri', 'tulang sendi', 'tulang nyeri', 'alami patah tulang', 'tulang kering kanan', 'bentur separator busway', 'kering kanan bentur']

Urologi
['bantu', 'perut', 'saran', 'samping', 'kandung', 'usg', 'rahim', 'kah', 'air', 'hasil', 'operasi', 'efek', 'ginjal', 'bak', 'kemih', 'urin', 'lambat', 'kelamin', 'kencing', 'gugur', 'batu', 'ml', 'perempuan', 'pasang', 'donor', 'urologi', 'testis', 'prostat', 'kateter', 'laser', 'beser', 'flow', 'salur kemih', 'kencing darah', 'sakit hasil', 'saran operasi', 'selang ginjal', 'usg ginjal', 'bedah urologi', 'pasang selang', 'batu ginjal', 'operasi batu', 'kandung kemih', 'retensi urin', 'ginjal operasi', 'operasi batu ginjal', 'batu ginjal operasi']
